# Terraform Cloud with AWS - Part 1

Teaser:
Join the ACME devops team on their journey to fully automated provisioning with AWS Cloud.

**Description:**

Work with the devops team at Acme Inc. as they go from manual provisioning with Terraform to a fully automated devops workflow with code reviews, testing, and automated provisioning in AWS Cloud. This workshop covers the following topics:

  * Terraform Open Source
  * Terraform Cloud
  * Remote State
  * Secure Variable Storage
  * Version Control Integration
  * Role-based Access Controls
  * Sentinel Security Policies
  * Collaboration on Changes
  * Private Module Registry
  * API Driven Workflows

## Pre-Requisites

* Terraform Cloud Account
	* Create a [free](https://app.terraform.io/signup/account) terraform cloud account.
* Create a Terraform organization.
	* Each person should have their own for this lab.
	* Name your new organization: "`YOURNAME-training`" (Replace with your actual name).
	* This organization name must be unique!
* AWS Credentials
* Visual Studio Code
* GitHub SaaS Account
	* Create a free one at  [https://github.com/signup](https://github.com/signup) .
	* We will be using GitHub for source control in this workshop.
* RECOMMENDATION: Anyone taking this training should already have experience using open source terraform. This training will not cover `terraform` or writing HCL. An understanding of Terraform open source usage will help in understanding the enterprise features. 

---

### Set Main Environment Variables

In [ ]:
export RED="\e[0;31m" YELLOW="\e[0;33m" BLDYELLOW="\e[1;33m" GREEN="\e[0;32m"
export CYAN="\e[0;36m" BLUE="\e[0;34m" WHITE="\e[0;37m" BLDWHITE="\e[1;37m"
export NC="\e[0m"

In [ ]:
# Set MAIN_DIR for future reference.
export MAIN_DIR=${PWD}
export GIT_DIR="config/terraform/hashicat-aws"
export TF_ORG="pphan"
export TF_WS_NAME="hashicat-aws"
echo $GIT_DIR
TFE_PROVIDER_DIR=config/terraform/tfe-provider

Some of these commands may place sensitive information. Prevent commands starting with a space to not be saved to shell history.

In [ ]:
HISTCONTROL=ignoreboth # do not save lines that begin with space in history

### AWS Credentials

`terraform` will use credentials set in your environment or through other means as described in the [Terraform documentation](https://www.terraform.io/docs/providers/aws/index.html#environment-variables). This guide will assume you are using the "**Environment Variables**" method.

Add your AWS credentials as two environment variables. Set your `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY` replacing `AAAAAA` with your own values.

In [ ]:
 export AWS_ACCESS_KEY_ID=AAAAAA
 export AWS_SECRET_ACCESS_KEY=AAAAAA
#  export AWS_SESSION_TOKEN=AAAAAA

NOTE: If you use `doormat`, then use this method. Be sure to replace `tf_org` and `ws_name` with your own values.

In [ ]:
tf_org="pphan"
ws_name="hashicat-aws"
account="se_demos_dev"
tf_hostname="app.terraform.io"

alias awstfcpush='doormat --smoke-test || doormat -r && doormat aws -a ${account} --tf-push --tf-workspace ${ws_name} --tf-organization ${tf_org} --tf-hostname ${tf_hostname}'
alias awscredsenv='doormat --smoke-test || doormat -r && eval $(doormat aws -a $account)'

awstfcpush
awscredsenv

Make sure your AWS Creds are set. Only showing half the values for security reasons.

In [ ]:
printf "${GREEN}# Make sure we have valid AWS credentials in our shell environment${NC}\n"
# echo $AWS_ACCESS_KEY_ID | grep -e '(?<![A-Z0-9])[A-Z0-9]{20}(?![A-Z0-9])' \
echo $AWS_ACCESS_KEY_ID | grep -Eo '[A-Z0-9]{11}' \
    || printf "${RED}Something went wrong. Unable to find a valid AWS_ACCESS_KEY_ID environment variable in your shell.${NC}\n"

# echo $AWS_SECRET_ACCESS_KEY | grep -e '(?<![A-Za-z0-9/+=])[A-Za-z0-9/+=]{40}(?![A-Za-z0-9/+=])' \
echo $AWS_SECRET_ACCESS_KEY | grep -Eo '[A-Za-z0-9/+=]{21}' \
    || printf "${RED}Something went wrong. Unable to find a valid AWS_SECRET_ACCESS_KEY environment variable in your shell.${NC}\n"

**Internal NOTE**: Using `--tf-push` to place variables in workspace. `doormat` locks down the IP so loading env vars into remote agent does not work. 

### Clone HashiCat Repo

In [ ]:
printf "${GREEN}#==> Clone hashicat-aws repo to config/terraform/hashicat-aws${NC}\n"
git clone https://github.com/hashicorp/hashicat-aws $GIT_DIR

# 🏡 Moving in - Set Up Your Workspace

**teaser:**

Configure your code editor for Terraform and open a workspace.

**notes:**

Welcome to your first day on the job at ACME Inc. These are some of your coworkers in the local ACME office:
* 👨🏻‍💼 Hiro - Product Manager
* 🧕🏽 Aisha - Database Admin
* 👮🏿‍♂️ William - InfoSec Lead
* 👨🏻‍🦲 Lars - Lead Developer
* 🧓🏻 Robin - Operations Admin
* 👩‍🎤 Jane - Quality Assurance
* 👳🏾‍♂️ Gaurav - Network Admin
* 👩🏼‍💼 Karen - Finance
* 🤓 You - Brand New Intern

Most modern text editors (ex Visual Studio Code, Atom, etc) support Terraform syntax highlighting.

## Assignment:
Welcome to your first day as an intern at ACME Inc. [ACME](https://www.youtube.com/watch?v=9m7evoFF83c) is a multi-national conglomerate that sells anvils, rocket powered roller skates, portable holes, earthquake pills and birdseed.

After employee orientation you sit down at your workstation and open your text editor, Visual Studio Code.

### Install Visual Studio Code

In [ ]:
# Install Visual Studio Code if you don't have it.
brew install --cask visual-studio-code

Launch Visual Studio Code or your text editor

In [ ]:
code .

NOTE: This will launch VS Code in the cloned directory.

### Install Terraform Extension

Next, install the Terraform extension to enable syntax highlighting in your code.

* Click on the "**Extensions**" icon - it looks like four small boxes. 
* Search for **HashiCorp Terraform** and be sure to select the "**Terraform 2.x.x**" extension from HashiCorp.
* Click the **Install** button to install the extension.
* Then click the **Reload Required** button to activate it.

Congratulations, you are ready to start working with Terraform on AWS. We'll use the `hashicat-aws` example app in the rest of the challenges as you learn new Terraform skills.

---

# ☁️ Terraform Cloud Setup

**teaser**:

Terraform Cloud offers unlimited free Terraform state storage for users. Safeguard your state files by storing them remotely in Terraform Cloud.

**notes:**

Terraform Cloud remote state storage is free for all users.

## assignment:

### Sign Up for Terraform Cloud

* Sign up for a free Terraform Cloud account at the following URL: 
	* https://app.terraform.io/signup/account
	* If you already have an account, then sign in with your existing credentials.
* New Accounts with no TFC Org
	* Once you're signed into Terraform Cloud, you'll see a screen with several options.
	* Click the "**Not right now, skip questions**" link and confirm that you want to skip the questions.
	* Create a new **organization** called `YOURNAME-training`. 
	* Replace `YOURNAME` with your own name or other text.

### Create a workspace with UI

Next you'll be prompted to create a workspace.

* Select the "**CLI-driven workflow**".
* Name your workspace `hashicat-aws`.
  * **NOTE**: ⚠️ You *MUST* name your workspace **`hashicat-aws`**. If you don't, the exercises will break. Do not attempt to name it something else.
* Click "**Create workspace**".

### Configure Your Workspace with UI

::**DO NOT SKIP THIS NEXT STEP:**::

#### Terraform Version

Make sure Terraform Version in Workspace matches the Terraform version on your local machine.
* Go to the TFC workspace's **Settings >> General** page. Note the "Terraform Version".
* Run `terraform version`.
* Change "**Terraform Version**" in you Workspace so that the two matches.

In [ ]:
terraform version

#### Terraform Execution Mode

Change the workspace execution mode to "**Local**".

* Go to the TFC workspace's **Settings >> General** page.
* Change the "**Execution Mode**" to **Local**.
* **NOTE:** This will allow us to perform Terraform runs from our workstation with local variables.
* Click **Save settings**. 

### Create and Configure a Workspace with TFE Provider

You can also create a workspace with the [TFE provider](https://registry.terraform.io/providers/hashicorp/tfe/latest/docs). The following steps will setup the following items via the TFE Provider:

* `execution_mode`
* `terraform_version`
* `auto_apply`
* `queue_all_runs`

**NOTE:** Don't do this section, if you have already created and configured a workspace with the UI above.

In [ ]:
TFE_PROVIDER_DIR=config/terraform/tfe-provider
TF_WS_NAME="hashicat-aws"
TF_ORG="pphan"

cd $MAIN_DIR
mkdir -p $TFE_PROVIDER_DIR

In [ ]:
cat <<-"EOF" > ${TFE_PROVIDER_DIR}/workspace-hashicat-aws.tf
terraform {
  required_providers {
    tfe = {
      source  = "hashicorp/tfe"
      version = "0.25.3"
    }
    random = {
      source  = "hashicorp/random"
      version = "3.1.0"
    }
  }
  required_version = "> 0.14"
}

# resource "random_pet" "learn" {}

resource "tfe_workspace" "vault" {
  name         = "${var.tfc_hashicat_aws_workspace_name}" #-${random_pet.learn.id}"
  organization = var.tfc_org

#   vcs_repo {
#     identifier         = var.vault_repo_name
#     oauth_token_id     = var.vcs_oauth_token_id
#     ingress_submodules = true
#   }

  queue_all_runs = false
  execution_mode = var.execution_mode
  auto_apply = var.auto_apply
  terraform_version = var.terraform_version
}

#// TFC Variables and Outputs
variable "auto_apply" {
  description = "automatically apply changes when a Terraform plan is successful"
  default     = false
}
variable "execution_mode" {
  description = "Which execution mode to use - remote, local, agent"
  default     = "local"
}
variable "tfc_org" {
  description = "The Terraform Cloud organization to create things in"
  default     = "REPLACE_ME"
}
variable "tfc_hashicat_aws_workspace_name" {
  description = "hashicat-aws workspace name"
  default     = "hashicat-aws"
}
variable "terraform_version" {
  description = "Version of Terraform to use for this workspace."
  default     = null
}

output "workspace_id" {
    value = tfe_workspace.vault.id
}

EOF

In [ ]:
tee ${TFE_PROVIDER_DIR}/terraform.tfvars <<-EOF
auto_apply                      = false
execution_mode                  = "local"
tfc_hashicat_aws_workspace_name = "${TF_WS_NAME}"
tfc_org                         = "${TF_ORG}"
terraform_version               = "$(terraform version -json | jq -r .terraform_version)"
EOF

In [ ]:
tf -chdir=${TFE_PROVIDER_DIR} init

In [ ]:
terraform -chdir=${TFE_PROVIDER_DIR} plan

In [ ]:
terraform -chdir=${TFE_PROVIDER_DIR} apply -auto-approve

### (optional) Destroy Workspace

See what you will destroy.

In [ ]:
printf "no\n" | terraform -chdir=config/terraform/tfe-provider destroy

Destroy your workspace

In [ ]:
terraform -chdir=config/terraform/tfe-provider destroy -auto-approve

### Enable 30 day trial of Terraform Cloud paid features

If you never used a trial with your org:

* Go to Organization: **Settings** > **Plan & Billing**.
* Select link for "**Click here to get started**".
* "**Trial Plan**" should be selected.
* Click "**Start your free trial**".

Or, if you have an existing account where you've already used a trial:

* Provide your instructor/SE with your **Organization's name**. 
* They can upgrade your organization to unlock a 30 day free trial of all paid features.

### Verification

> ⚠️  Did you change your **Execution Mode** to **Local**? This is an easy step to miss, so we mention it twice.

---

# 📒 Safekeeping Your Terraform State

**Teaser:**

An unexpected outage has taken down one of the production websites. It took longer than expected to recover because the Terraform state file was stored on someone's laptop. Terraform Cloud's remote state feature is here to help.

**Notes:**

It's Monday morning and you're the first one into the office. Most of your teammates were up late fixing last night's outage. Eventually senior operations admin Robin shows up at your desk.

🧓 Hey kiddo, how are you doing? Listen, I want your help with something. Last night we had trouble rebuilding the website because the Terraform state file was stored on Lars' laptop. And guess what, Lars is on vacation for the next two weeks. Why don't you help me configure remote state on this application so this doesn't happen again?

## assignment:

Configure remote state using your Terraform Cloud account. In order to complete this challenge you'll need the following:

1. A free Terraform Cloud account. Log in at https://app.terraform.io
2. An organization called `<YOURNAME>-training`. You created this in the previous exercise.
3. A workspace named `hashicat-aws` with its **Execution Mode** set to **Local** (NOT Remote)
4. A Terraform **user** API token for authentication
5. A `remote_backend` config stored in your workspace

### Generate TFC User API Token

**`terraform login` method**

* Use the `terraform login` command to generate and save your token to a local config file.
    * **NOTE:** Run this in a separate terminal window, since Jupyter does not allow interactive commands.
* Click on the link to open the TFC tokens page. Sample Output

```shell
Terraform must now open a web browser to the tokens page for app.terraform.io.

If a browser does not open this automatically, open the following URL to proceed:
	https://app.terraform.io/app/settings/tokens?source=terraform-login
```

* For **Description:** Accept default or provide your own value.
* Click **Create API token**.
* Copy the token.
	* NOTE: Don't click **Done** until after the token is setup successfully.
* Go back to separate terminal window.
* Paste the password.
* **NOTE**: You won't see your token appear on the screen when you paste it into the terminal.
* Press <kbd>Enter</kbd>

Alternatively

* Go directly to this link in a new browser tab:
https://app.terraform.io/app/settings/tokens?source=terraform-login
* Create your token.
* Paste the value into the `TF_TOKEN` variable below and run the cell.

In [ ]:
mkdir -p ${HOME}/.terraform.d
TF_TOKEN="YOURTOKEN"
cat > ${HOME}/.terraform.d/credentials.tfrc.json <<-EOF
{
  "credentials": {
    "app.terraform.io": {
      "token": "${TF_TOKEN}"
    }
  }
}
EOF

Verify that your token is stored in the `${HOME}/.terraform.d/credentials.tfrc.json` file.

In [ ]:
cat ${HOME}/.terraform.d/credentials.tfrc.json

### Configure a remote backend

Create `remote_backend` file.

* Replace the `TF_ORG` variable with your organization name
* Replace the `TF_WS` variable with your workspace name.

Sample `remote_backend.tf`

```go
terraform {
  backend "remote" {
    hostname = "app.terraform.io"
    organization = "peterphan"
    workspaces {
      name = "hashicat-aws"
    }
  }
}
```

> NOTE: Do **NOT** delete this file. It is required to complete the track successfully.

In [ ]:
# export TF_WS=hashicat-aws

Go to back to `$MAIN_DIR`.

In [ ]:
cd $MAIN_DIR

In [ ]:
tee ${GIT_DIR}/remote_backend.tf <<-EOF
terraform {
  backend "remote" {
    hostname = "app.terraform.io"
    organization = "${TF_ORG}"
    workspaces {
      name = "${TF_WS_NAME}"
    }
  }
}
EOF

### Set Variables with terraform.tfvars file

* Edit/create the "`terraform.tfvars`" file.
* Set `prefix` to your name (first and last with or without a hyphen between them and all lower case).
* Set `region` to a valid AWS region such as "`us-east-1`", "`us-west-2`", "`eu-west-2`", or "`ap-southeast-1`".
    * See this [page](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/using-regions-availability-zones.html#concepts-available-regions) for a list of valid AWS regions.

**NOTE**: The variables are declared in the "`variables.tf`" file. The "`terraform.tfvars`" file is used to set values for them.
**NOTE:** Do not `us-west-2`. One of its AZs do not support the `t2.micro` instance type.

In [ ]:
cat <<-EOF > $GIT_DIR/terraform.tfvars
instance_type = "t2.micro"
prefix = "pphan"
region = "us-east-1"
EOF

### Deploy hashicat

You should have a token stored at `${HOME}/.terraform.d/credentials.tfrc.json` and a remote backend configured. 

* Initialize terraform. Use `-migrate-state` to copy existing state to new backend.

In [ ]:
terraform -chdir=${GIT_DIR} init -migrate-state 

* Confirm (type `yes`) that you want to copy the state file to TFC workspace.
* **NOTE**: If you ran terraform locally before configuring remote state, you’ll have a local state file called `terraform.tfstate`. 
  * Delete the local state file if it exists. `init` has already moved existing state to TFC.

In [ ]:
# if needed, removing existing local state file
rm ${GIT_DIR}/terraform.tfstate

Run `terraform apply` command. We use `-auto-approve` to avoid being prompted.

In [ ]:
terraform -chdir=${GIT_DIR} apply -auto-approve

* When the `terraform apply` finishes, you should see output like this:

```shell
Outputs:
catapp_ip = "http://44.237.220.215"
catapp_url = "http://ec2-44-237-220-215.us-west-2.compute.amazonaws.com"
```

### Verification

* Confirm that terraform has run successfully and the **hashicat** app is working.
    * Click on the `catapp_url` to confirm your application is working.
    * If it won't load, please run the `terraform apply` command again. That usually fixes this problem.

Open a browser tab to the `catapp_url`.

In [ ]:
open $(terraform -chdir=${GIT_DIR} output -raw catapp_url)

You should see a new state file on the "**States**" tab of your Terraform Cloud workspace.

In [ ]:
echo "#==> Go to https://app.terraform.io/app/${TF_ORG}/workspaces/${TF_WS_NAME}/states"
open https://app.terraform.io/app/${TF_ORG}/workspaces/${TF_WS_NAME}/states

You've successfully deployed the hashicat application with remote state enabled.

### (optional) Destroy hashicat-aws

If you no longer need or you want to start over, you can destroy your resources.

In [ ]:
terraform -chdir=${GIT_DIR} destroy -auto-approve

# 🔐 Securing Cloud Credentials

**teaser**:

Your team has started building cloud infrastructure on AWS, but the security
    team is concerned about protecting access to everyone's cloud credentials.

**notes**:
<details><summary></summary>

After a few weeks on the job you're starting to get into the rhythm of things. Write some code, run some tests, deploy the website. Everything's going great until someone's AWS keys are accidentally pushed to a public code repository. You get this email from William, the lead infosec admin at ACME:

> 👮🏿‍♂️ Hello junior admin, we ran a remote scan on your laptop last night and found some unsecured AWS access keys. We need you to move those off your laptop and store them in Terraform Cloud by the end of the day.

🤔 Did you know?

Thousands of API and cryptographic keys are leaking on GitHub every day!

https://nakedsecurity.sophos.com/2019/03/25/thousands-of-coders-are-leaving-their-crown-jewels-exposed-on-github/

When you store your API keys as sensitive variables they are encrypted and stored in an instance of HashiCorp Vault. These keys are only decrypted in a trusted, secure container that runs the Terraform command.

👩🏼‍💻 Remote Execution, Local Code

**Remote Execution** allows you to use the same Terraform commands that you're familiar with, but the run and all your variables are safely stored in your Terraform Cloud workspace. This can be helpful when you're upgrading tools that were originally written for Terraform Open Source.

With Remote Execution your Terraform code is still stored on your local machine and sent to the server each time you run.
</details>

## assignment:

After the AWS credentials issue, the security team is tightening down access to your AWS account. API keys must now be secured as stored variables in Terraform Cloud. Your task is to find your **AWS Access Key ID** and **Secret Access Key**, and move them into your workspace as secure environment variables.

You'll store the Access Key ID as a plain text environment variable, and the Secret Access Key as a **sensitive** environment variable. You may also enter optional descriptions for each variable but this is not required to complete the challenge.

> NOTE: Do not set environment variables, if you used `doormat` with `--tf-push`.

1. Find your AWS credentials, which you set earlier.

In [ ]:
echo $AWS_ACCESS_KEY_ID
echo $AWS_SECRET_ACCESS_KEY

2. Or, you might have your AWS credentials stored in `${HOME}/.aws/credentials`.

In [ ]:
# export AWS_ACCESS_KEY_ID=$(grep -A2 default ~/.aws/credentials | awk '/access_key_id/{print $NF}')
# export AWS_SECRET_ACCESS_KEY=$(grep -A2 default ~/.aws/credentials | awk '/secret_access_key/{print $NF}')
grep -A2 default ~/.aws/credentials | awk '/access_key_id/{print $NF}'
grep -A2 default ~/.aws/credentials | awk '/secret_access_key/{print $NF}'

### Configure General Settings for TFC Workspace with UI

3. Update the **Execution Mode** settings in your workspace to **Remote** on the **Settings > General tab**.
1. Change the **Apply Method** to **Auto apply**. 
	* This will save you the trouble of having to approve every Terraform run manually. 
	* Click the **Save** button at the bottom of the page!

#### (optional) Modify TFC Workspace with Terraform Provider

Instead of using the UI, you can modify your Workspace using the Terraform Provider.

Change the `auto_apply` and `execution_mode`.

In [ ]:
tee ${TFE_PROVIDER_DIR}/terraform.tfvars <<-EOF
auto_apply                      = true
execution_mode                  = "remote"
tfc_hashicat_aws_workspace_name = "${TF_WS_NAME}"
tfc_org                         = "${TF_ORG}"
terraform_version               = "$(terraform version -json | jq -r .terraform_version)"
EOF

In [ ]:
terraform -chdir=config/terraform/tfe-provider plan

In [ ]:
terraform -chdir=config/terraform/tfe-provider apply -auto-approve

### Configure Variables for TFC Workspace

We will cover creating Terraform Workspace Variables in two general ways 1) UI and 2) Provider, API, `tf-helper`.

#### Configure Variables in TFC Workspace with UI

* Go to the **Variables** tab.
* Set **Environment Variables** for your AWS credentials.
	* Store the **`AWS_ACCESS_KEY_ID`** as a regular environment variable.
	* Mark the **`AWS_SECRET_ACCESS_KEY`** as sensitive.
* Set **Terraform Variables** for your `prefix` and `region`. 
	* Learn more about these variables by looking in the `variables.tf` file.
    * Set the same values you used in your `terraform.tfvars` file to avoid all the resources being destroyed and re-created.
	* **NOTE**: You **must** configure these variables on the remote workspace, as they will no longer be read from your local `terraform.tfvars` file.

> **TIP**
> This code has been tested in the **us-east-1** region. Some regions do not support the `t2.micro` instance type. If you encounter this, switch your region to `us-east-1`.

The UI is great and simple for a handful of workspaces. However, if you have to change these values many times and/or for many workspaces, then you want to consider the TFE Provider, API, or utilities like `tf-helper`.

#### (optional) Create TFC Workspace Terraform Variables with TFE Provider

You can create Workspace variables using the Terraform Provider. We will use the provider to create Terraform variables for `prefix` and `region`.

In [ ]:
cat <<-"EOF" > config/terraform/tfe-provider/workspace-hashicat-aws-vars.tf
## Vault Variables
resource "tfe_variable" "prefix" {
  key          = "prefix"
  value        = var.prefix
  category     = "terraform"
  workspace_id = tfe_workspace.vault.id
  description  = "Workspace that created the Kubernetes k8s"
}

resource "tfe_variable" "region" {
  key          = "region"
  value        = var.region
  category     = "terraform"
  workspace_id = tfe_workspace.vault.id
  description  = "Workspace that created the Consul"
  sensitive    = false
}

variable "prefix" {
  description = "The Terraform Cloud organization to create things in"
  default = "pphan"
}
variable "region" {
  description = "The Terraform Cloud organization to create things in"
  default = "us-east-1"
}
EOF

In [ ]:
terraform -chdir=config/terraform/tfe-provider apply -auto-approve

#### (optional) Create TFC Workspace Environment Variables with TFC API

Instead of using the UI or Terraform Provider, you can create/update TFC variables using the API. We will use the API with `curl` to create TFC Workspace variables for `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY`.

In [ ]:
# export TF_VAR_aws_access_key_id=${AWS_ACCESS_KEY_ID}
# export TF_VAR_aws_secret_access_key=${AWS_SECRET_ACCESS_KEY}

export TF_ADDR=https://app.terraform.io
# Get the ID of the workspace.
export TF_WS_ID=$(terraform -chdir=config/terraform/tfe-provider output -raw workspace_id)
export TF_TOKEN=$(jq -r '.credentials."app.terraform.io".token' ${HOME}/.terraform.d/credentials.tfrc.json)
echo $TF_WS_ID

(optional) Delete the workspace variables. Skip this if you have not created AWS env vars. If they already exist, you can't just overwrite them. They will need to be deleted first.

In [ ]:
printf "${GREEN}# Get list of all variables in the workspace${NC}\n"
list_variables_result=$(curl -s --header "Authorization: Bearer $TF_TOKEN" --header "Content-Type: application/vnd.api+json" "${TF_ADDR}/api/v2/vars?filter%5Borganization%5D%5Bname%5D=${TF_ORG}&filter%5Bworkspace%5D%5Bname%5D=${TF_WS_NAME}" | jq -c)
# debugging - uncomment next line
# echo $list_variables_result | jq -c

printf "\n${GREEN}# Create bash function to extract variable IDs and names with python${NC}\n"
parse_ids_and_names() { python -c '
import sys, json
parsed = json.load(sys.stdin)
id_name_category_dict = ",".join([v["id"] + ":" + v["attributes"]["key"] + ":" + v["attributes"]["category"] for v in parsed["data"] if v["attributes"]["key"].startswith("AWS_")])
print(id_name_category_dict)'
}
printf "\n${YELLOW}# We filtered for variables that start with AWS_${NC}\n"

printf "\n${GREEN}# Parse variables from list_variables_result${NC}\n"
variables_map=$(echo $list_variables_result | parse_ids_and_names)
echo ${variables_map} #debugging

printf "\n${GREEN}# Delete AWS variables in workspace${NC}\n"
for v in $(echo $variables_map | sed "s/,/ /g")
do
    # Separate ID, name, and category
    v_id=$(echo $v | cut -f1 -d":")
    v_name=$(echo $v | cut -f2 -d":")
    v_category=$(echo $v | cut -f3 -d":")

    # Delete variable
    echo "Deleting ${v_category} variable ${v_name}"
    curl -s -H "Authorization: Bearer ${TF_TOKEN}" -H "Content-Type: application/vnd.api+json" -X DELETE "${TF_ADDR}/api/v2/vars/${v_id}"
done

Use the API with `curl` to create TFC Workspace variables for `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY`.

In [ ]:
printf "${GREEN}#==> Set AWS access key${NC}\n"
cat <<EOF | curl -H "Authorization: Bearer ${TF_TOKEN}" -H "Content-Type: application/vnd.api+json" -X POST -d @- ${TF_ADDR}/api/v2/workspaces/${TF_WS_ID}/vars 
{
  "data": {
    "type":"vars",
    "attributes": {
      "key":"AWS_ACCESS_KEY_ID",
      "value":"${AWS_ACCESS_KEY_ID}",
      "category":"env",
      "sensitive":false
    }
  }
}
EOF

printf "\n\n${GREEN}#==> Set AWS secret key${NC}\n"
cat <<EOF | curl -H "Authorization: Bearer ${TF_TOKEN}" -H "Content-Type: application/vnd.api+json" -X POST -d @- ${TF_ADDR}/api/v2/workspaces/${TF_WS_ID}/vars
{
  "data": {
    "type":"vars",
    "attributes": {
      "key":"AWS_SECRET_ACCESS_KEY",
      "value":"${AWS_SECRET_ACCESS_KEY}",
      "category":"env",
      "sensitive":true
    }
  }
}
EOF

printf "\n\n${GREEN}#==> Set AWS session token${NC}\n"
cat <<EOF | curl -H "Authorization: Bearer ${TF_TOKEN}" -H "Content-Type: application/vnd.api+json" -X POST -d @- ${TF_ADDR}/api/v2/workspaces/${TF_WS_ID}/vars
{
  "data": {
    "type":"vars",
    "attributes": {
      "key":"AWS_SESSION_TOKEN",
      "value":"${AWS_SESSION_TOKEN}",
      "category":"env",
      "sensitive":true
    }
  }
}
EOF

#### (optional) Create TFC Workspace Environment Variables with tf-helper
Instead of using the UI or Terraform Provider, you can create/update TFC variables using the `tf-helper` utility. We will overwrite the AWS Workspace Environment Variables.

Install `tf-helper`.

In [ ]:
wget https://raw.githubusercontent.com/hashicorp-community/tf-helper/release/tfh/bin/tfh
git clone git@github.com:hashicorp-community/tf-helper.git
pushd tf-helper/tfh/bin
ln -s $PWD/tfh /usr/local/bin/tfh
popd

Set environment variables that `tf-helper` requires.

In [ ]:
export TFH_token=$TF_TOKEN
export TFH_org=$TF_ORG
echo $TFH_org
export TFH_name=$TF_WS_NAME
echo $TFH_name

In [ ]:
tfh pushvars -var AWS_ACCESS_KEY_ID=$AWS_ACCESS_KEY_ID -svar AWS_SECRET_ACCESS_KEY=${AWS_SECRET_ACCESS_KEY} -vvv -overwrite-all

#### Optional list variables using API

In [ ]:
curl -H "Authorization: Bearer ${TF_TOKEN}" -H "Content-Type: application/vnd.api+json" \
    "${TF_ADDR}/api/v2/vars?filter%5Borganization%5D%5Bname%5D=${TF_ORG}&filter%5Bworkspace%5D%5Bname%5D=${TF_WS_NAME}" \
    | jq -rc '.data[]'

### Verify

* TIP: If you used `doormat` to set your AWS credentials, then changed it with the steps above, you will need to reset with `doormat`.

In [ ]:
awstfcpush

* Verify by running `terraform init`. Your backend configuration will be updated for remote execution.

In [ ]:
terraform -chdir=${GIT_DIR} init

* Next, try running `terraform plan`. 
	* This will trigger what's known as a speculative plan. 
	* You can view this plan by clicking the link from your terminal.
    * Speculative plans don't show up under the **Runs** tab.

In [ ]:
terraform -chdir=${GIT_DIR} plan

```shell
Preparing the remote plan...

To view this run in a browser, visit:
https://app.terraform.io/app/pphan/hashicat-aws/runs/run-WM1UuA6ytUH7huLT
```
	* 📓 This plan will not show up in your TFC **Runs** tab that are triggered via the UI or API.
	* A copy of the output will be streamed back to your terminal. 
* Run a `terraform apply` to see it in action:
	* The apply will show up if you navigate to the runs page in the Terraform Cloud UI.

In [ ]:
terraform -chdir=${GIT_DIR} apply -auto-approve

### Check Securing Cloud Credentials

This section is OPTIONAL but useful if you are running into problems.

Below is a set of commands to check if the steps above were all performed. This highlights the power of the Terraform Cloud API.

In [ ]:
printf "${GREEN}# Test whether remote exec is enabled on the workspace.${NC}\n"
REMOTE_EXEC_ENABLED=$(curl -s --header "Authorization: Bearer $TF_TOKEN" --header "Content-Type: application/vnd.api+json" --request GET https://app.terraform.io/api/v2/organizations/$TF_ORG/workspaces/${TF_WS_NAME} | jq .data.attributes.operations)
# echo $REMOTE_EXEC_ENABLED
[[ $REMOTE_EXEC_ENABLED == true ]] || echo "Oops, it looks like remote exec is not enabled on your workspace."

printf "${GREEN}# Does the workspace have an AWS_ACCESS_KEY_ID${NC}\n"
PLAINTEXT_ID=$(curl -s --header "Authorization: Bearer $TF_TOKEN" --header "Content-Type: application/vnd.api+json" "https://app.terraform.io/api/v2/vars?filter%5Borganization%5D%5Bname%5D=$TF_ORG&filter%5Bworkspace%5D%5Bname%5D=$TF_WS_NAME"  | jq -r '.data | .[] | select(.attributes.key|test("AWS_ACCESS_KEY_ID")) | .attributes.key')

[[ $PLAINTEXT_ID == "AWS_ACCESS_KEY_ID" ]] || echo "Uh oh, we didn't find an environment variable for your AWS_ACCESS_KEY_ID. Try again."
printf "${GREEN}# Does the workspace have a secure AWS_SECRET_ACCESS_KEY${NC}\n"
SECURE_KEY=$(curl -s --header "Authorization: Bearer $TF_TOKEN" --header "Content-Type: application/vnd.api+json" "https://app.terraform.io/api/v2/vars?filter%5Borganization%5D%5Bname%5D=$TF_ORG&filter%5Bworkspace%5D%5Bname%5D=$TF_WS_NAME" | jq '.data | .[] | select(.attributes.key|test("AWS_SECRET_ACCESS_KEY")) | .attributes.sensitive')

[[ $SECURE_KEY == true ]] || echo "Uh oh, we didn't find a secure variable for your AWS_ACCESS_KEY_ID. Try again."

printf "${GREEN}# Does the workspace have a prefix variable${NC}\n"
curl -s --header "Authorization: Bearer $TF_TOKEN"   --header "Content-Type: application/vnd.api+json" "https://app.terraform.io/api/v2/vars?filter%5Borganization%5D%5Bname%5D=$TF_ORG&filter%5Bworkspace%5D%5Bname%5D=$TF_WS_NAME" | grep -q "prefix" || fail-message "Oh dear, it looks like you don't have a Terraform variable called 'prefix' in your workspace."

printf "${GREEN}# Does the workspace have a region variable${NC}\n"
curl -s --header "Authorization: Bearer $TF_TOKEN"   --header "Content-Type: application/vnd.api+json" "https://app.terraform.io/api/v2/vars?filter%5Borganization%5D%5Bname%5D=$TF_ORG&filter%5Bworkspace%5D%5Bname%5D=$TF_WS_NAME" | grep -q "region" || fail-message "Oh dear, it looks like you don't have a Terraform variable called 'region' in your workspace."

Congratulations, your AWS keys are now safely encrypted and stored in your Terraform Cloud workspace.

You can continue to run `terraform plan` and `terraform apply` in your terminal, but the execution is now done in Terraform Cloud.

---

## Next Steps

If you no longer need this environment, proceed to the Clean Up section. Or you can go to the next part [Terraform Cloud AWS Part 2](Terraform_Cloud_AWS_2.ipynb).

# Clean Up

### Destroy hashicat-aws

In [ ]:
cd $MAIN_DIR; pwd

In [ ]:
terraform -chdir=config/terraform/hashicat-aws destroy

### Destroy hashicat-aws workspace

In [ ]:
terraform -chdir=config/terraform/tfe-provider destroy -auto-approve

### Destroy hashicat-aws artifacts

Destroy the cloned `hashicat-aws` directory and the `tfe-provider` directory.

In [ ]:
rm -rf config/terraform/hashicat-aws
rm -rf config/terraform/tfe-provider